In [ ]:
#!/usr/bin/python3
# docdist1 - initial version of document distance
# docdist2 - changed concatenate to extend in get_words_from_line_list
# docdist3 - improved dot product to exploit sorted order and achieve
#               linear instead of quadratic time
# docdist4 - changed count_frequency to use dictionaries instead of lists
# docdist5 - change get_words_from_string to use string translate and split
# docdist6 - changed sorting from insertion sort to merge sort
# docdist7 - remove sorting altogether via more hashing
# docdist8 - treat whole file as a single "line"
# 
#
# Original version by Ronald L. Rivest on February 14, 2007
# Revision by Erik D. Demaine on September 12, 2011
#
# Usage:
#    filename.py filename1 filename2
#     
# This program computes the "distance" between two text files
# as the angle between their word frequency vectors (in radians).
#
# For each input file, a word-frequency vector is computed as follows:
#    (1) the specified file is read in
#    (2) it is converted into a list of alphanumeric "words"
#        Here a "word" is a sequence of consecutive alphanumeric
#        characters.  Non-alphanumeric characters are treated as blanks.
#        Case is not significant.
#    (3) for each word, its frequency of occurrence is determined
#    (4) the word/frequency lists are sorted into order alphabetically
#
# The "distance" between two vectors is the angle between them.
# If x = (x1, x2, ..., xn) is the first vector (xi = freq of word i)
# and y = (y1, y2, ..., yn) is the second vector,
# then the angle between them is defined as:
#    d(x,y) = arccos(inner_product(x,y) / (norm(x)*norm(y)))
# where:
#    inner_product(x,y) = x1*y1 + x2*y2 + ... xn*yn
#    norm(x) = sqrt(inner_product(x,x))

In [55]:
if __name__ == "__main__":
    import profile
    profile.run("ver8()")

File pg-grimm.txt : 540174 lines, 105324 words, 5172 distinct words
File pg-huckleberry_finn.txt : 594262 lines, 120896 words, 6519 distinct words
The distance between the documents is: 0.460007 (radians)
         554 function calls in 0.094 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 :0(_getdefaultlocale)
        1    0.000    0.000    0.000    0.000 :0(acos)
       39    0.000    0.000    0.000    0.000 :0(acquire)
       39    0.000    0.000    0.000    0.000 :0(append)
        2    0.000    0.000    0.000    0.000 :0(charmap_decode)
        1    0.000    0.000    0.094    0.094 :0(exec)
       38    0.000    0.000    0.000    0.000 :0(getpid)
       38    0.000    0.000    0.000    0.000 :0(isinstance)
        6    0.000    0.000    0.000    0.000 :0(len)
        2    0.000    0.000    0.000    0.000 :0(open)
        9    0.000    0.000    0.000    0.000 :0(print)
      

In [2]:
import math
import sys
import string

In [3]:
def read_file(filename):
    """ 
    Read the text file with the given filename;
    return a list of the lines of text in the file.
    """
    try:
        f = open(filename, 'r')
        return f.readlines()
    except IOError:
        print("Error opening or reading input file: ",filename)
        sys.exit()

In [4]:
def get_words_from_line_list(L):
    """
    Parse the given list L of text lines into words.
    Return list of all words found.
    """

    word_list = []
    for line in L:
        words_in_line = get_words_from_string(line)
        word_list = word_list + words_in_line
    return word_list

In [5]:
def get_words_from_string(line):
    """
    Return a list of the words in the given input string,
    converting each word to lower-case.

    Input:  line (a string)
    Output: a list of strings 
              (each string is a sequence of alphanumeric characters)
    """
    word_list = []          # accumulates words in line
    character_list = []     # accumulates characters in word
    for c in line:
        if c.isalnum():
            character_list.append(c)
        elif len(character_list)>0:
            word = "".join(character_list)
            word = word.lower()
            word_list.append(word)
            character_list = []
    if len(character_list)>0:
        word = "".join(character_list)
        word = word.lower()
        word_list.append(word)
    return word_list

In [6]:
def count_frequency(word_list):
    """
    Return a list giving pairs of form: (word,frequency)
    """
    L = []
    for new_word in word_list:
        for entry in L:
            if new_word == entry[0]:
                entry[1] = entry[1] + 1
                break
        else:
            L.append([new_word,1])
    return L

In [7]:
def insertion_sort(A):
    """
    Sort list A into order, in place.

    From Cormen/Leiserson/Rivest/Stein,
    Introduction to Algorithms (second edition), page 17,
    modified to adjust for fact that Python arrays use 
    0-indexing.
    """
    for j in range(len(A)):
        key = A[j]
        # insert A[j] into sorted sequence A[0..j-1]
        i = j-1
        while i>-1 and A[i]>key:
            A[i+1] = A[i]
            i = i-1
        A[i+1] = key
    return A

In [8]:
def word_frequencies_for_file(filename):
    """
    Return alphabetically sorted list of (word,frequency) pairs 
    for the given file.
    """

    line_list = read_file(filename)
    word_list = get_words_from_line_list(line_list)
    freq_mapping = count_frequency(word_list)
    insertion_sort(freq_mapping)

    print("File",filename,":", end=' ')
    print(len(line_list),"lines,", end=' ')
    print(len(word_list),"words,", end=' ')
    print(len(freq_mapping),"distinct words")

    return freq_mapping

In [9]:
def inner_product(L1,L2):
    """
    Inner product between two vectors, where vectors
    are represented as lists of (word,freq) pairs.

    Example: inner_product([["and",3],["of",2],["the",5]],
                           [["and",4],["in",1],["of",1],["this",2]]) = 14.0 
    """
    sum = 0.0
    for word1, count1 in L1:
        for word2, count2 in L2:
            if word1 == word2:
                sum += count1 * count2
    return sum

In [10]:
def vector_angle(L1,L2):
    """
    The input is a list of (word,freq) pairs, sorted alphabetically.

    Return the angle between these two vectors.
    """
    numerator = inner_product(L1,L2)
    denominator = math.sqrt(inner_product(L1,L1)*inner_product(L2,L2))
    return math.acos(numerator/denominator)

In [11]:
def ver1():
    filename_1 = "pg-grimm.txt"
    filename_2 = "pg-huckleberry_finn.txt"
    sorted_word_list_1 = word_frequencies_for_file(filename_1)
    sorted_word_list_2 = word_frequencies_for_file(filename_2)
    distance = vector_angle(sorted_word_list_1,sorted_word_list_2)
    print("The distance between the documents is: %0.6f (radians)"%distance)

In [12]:
# docdist2 - changed concatenate to extend in get_words_from_line_list

In [13]:
def get_words_from_line_list_ver2(L):
    """
    Parse the given list L of text lines into words.
    Return list of all words found.
    """

    word_list = []
    for line in L:
        words_in_line = get_words_from_string(line)
        # Using "extend" is much more efficient than concatenation here:
        word_list.extend(words_in_line)
    return word_list

In [14]:
def word_frequencies_for_file_ver2(filename):
    """
    Return alphabetically sorted list of (word,frequency) pairs 
    for the given file.
    """

    line_list = read_file(filename)
    word_list = get_words_from_line_list_ver2(line_list)
    freq_mapping = count_frequency(word_list)
    insertion_sort(freq_mapping)

    print("File",filename,":", end=' ')
    print(len(line_list),"lines,", end=' ')
    print(len(word_list),"words,", end=' ')
    print(len(freq_mapping),"distinct words")

    return freq_mapping

In [15]:
def ver2():
    filename_1 = "pg-grimm.txt"
    filename_2 = "pg-huckleberry_finn.txt"
    sorted_word_list_1 = word_frequencies_for_file_ver2(filename_1)
    sorted_word_list_2 = word_frequencies_for_file_ver2(filename_2)
    distance = vector_angle(sorted_word_list_1,sorted_word_list_2)
    print("The distance between the documents is: %0.6f (radians)"%distance)

In [16]:
# docdist3 - improved dot product to exploit sorted order and achieve
#            linear instead of quadratic time

In [17]:
def inner_product_ver3(L1,L2):
    """
    Inner product between two vectors, where vectors
    are represented as alphabetically sorted (word,freq) pairs.

    Example: inner_product([["and",3],["of",2],["the",5]],
                        3   [["and",4],["in",1],["of",1],["this",2]]) = 14.0 
    """
    sum = 0.0
    i = 0
    j = 0
    while i<len(L1) and j<len(L2):
        # L1[i:] and L2[j:] yet to be processed
        if L1[i][0] == L2[j][0]:
            # both vectors have this word
            sum += L1[i][1] * L2[j][1]
            i += 1
            j += 1
        elif L1[i][0] < L2[j][0]:
            # word L1[i][0] is in L1 but not L2
            i += 1
        else:
            # word L2[j][0] is in L2 but not L1
            j += 1
    return sum

In [18]:
def vector_angle_ver3(L1,L2):
    """
    The input is a list of (word,freq) pairs, sorted alphabetically.

    Return the angle between these two vectors.
    """
    numerator = inner_product_ver3(L1,L2)
    denominator = math.sqrt(inner_product_ver3(L1,L1)*inner_product_ver3(L2,L2))
    return math.acos(numerator/denominator)

In [19]:
def ver3():
    filename_1 = "pg-grimm.txt"
    filename_2 = "pg-huckleberry_finn.txt"
    sorted_word_list_1 = word_frequencies_for_file_ver2(filename_1)
    sorted_word_list_2 = word_frequencies_for_file_ver2(filename_2)
    distance = vector_angle_ver3(sorted_word_list_1,sorted_word_list_2)
    print("The distance between the documents is: %0.6f (radians)"%distance)

In [20]:
# docdist4 - changed count_frequency to use dictionaries instead of lists

In [21]:
def count_frequency_ver4(word_list):
    """
    Return a list giving pairs of form: (word,frequency)
    """
    D = {}
    for new_word in word_list:
        if new_word in D:
            D[new_word] = D[new_word]+1
        else:
            D[new_word] = 1
    return list(D.items())

In [22]:
def word_frequencies_for_file_ver4(filename):
    """
    Return alphabetically sorted list of (word,frequency) pairs 
    for the given file.
    """

    line_list = read_file(filename)
    word_list = get_words_from_line_list_ver2(line_list)
    freq_mapping = count_frequency_ver4(word_list)
    insertion_sort(freq_mapping)

    print("File",filename,":", end=' ')
    print(len(line_list),"lines,", end=' ')
    print(len(word_list),"words,", end=' ')
    print(len(freq_mapping),"distinct words")

    return freq_mapping

In [23]:
def ver4():
    filename_1 = "pg-grimm.txt"
    filename_2 = "pg-huckleberry_finn.txt"
    sorted_word_list_1 = word_frequencies_for_file_ver4(filename_1)
    sorted_word_list_2 = word_frequencies_for_file_ver4(filename_2)
    distance = vector_angle_ver3(sorted_word_list_1,sorted_word_list_2)
    print("The distance between the documents is: %0.6f (radians)"%distance)

In [24]:
# docdist5 - change get_words_from_string to use string translate and split

In [25]:
# global variables needed for fast parsing
# translation table maps upper case to lower case and punctuation to spaces
translation_table = str.maketrans(string.punctuation+string.ascii_uppercase,
                                     " "*len(string.punctuation)+string.ascii_lowercase)

def get_words_from_string_ver5(line):
    """
    Return a list of the words in the given input string,
    converting each word to lower-case.

    Input:  line (a string)
    Output: a list of strings 
              (each string is a sequence of alphanumeric characters)
    """
    line = line.translate(translation_table)
    word_list = line.split()
    return word_list

In [26]:
def get_words_from_line_list_ver5(L):
    """
    Parse the given list L of text lines into words.
    Return list of all words found.
    """

    word_list = []
    for line in L:
        words_in_line = get_words_from_string_ver5(line)
        # Using "extend" is much more efficient than concatenation here:
        word_list.extend(words_in_line)
    return word_list

In [27]:
def word_frequencies_for_file_ver5(filename):
    """
    Return alphabetically sorted list of (word,frequency) pairs 
    for the given file.
    """

    line_list = read_file(filename)
    word_list = get_words_from_line_list_ver5(line_list)
    freq_mapping = count_frequency_ver4(word_list)
    insertion_sort(freq_mapping)

    print("File",filename,":", end=' ')
    print(len(line_list),"lines,", end=' ')
    print(len(word_list),"words,", end=' ')
    print(len(freq_mapping),"distinct words")

    return freq_mapping

In [28]:
def ver5():
    filename_1 = "pg-grimm.txt"
    filename_2 = "pg-huckleberry_finn.txt"
    sorted_word_list_1 = word_frequencies_for_file_ver5(filename_1)
    sorted_word_list_2 = word_frequencies_for_file_ver5(filename_2)
    distance = vector_angle_ver3(sorted_word_list_1,sorted_word_list_2)
    print("The distance between the documents is: %0.6f (radians)"%distance)

In [29]:
# docdist6 - changed sorting from insertion sort to merge sort

In [30]:
def merge_sort(A):
    """
    Sort list A into order, and return result.
    """
    n = len(A)
    if n==1: 
        return A
    mid = n//2     # floor division
    L = merge_sort(A[:mid])
    R = merge_sort(A[mid:])
    return merge(L,R)

def merge(L,R):
    """
    Given two sorted sequences L and R, return their merge.
    """
    i = 0
    j = 0
    answer = []
    while i<len(L) and j<len(R):
        if L[i]<R[j]:
            answer.append(L[i])
            i += 1
        else:
            answer.append(R[j])
            j += 1
    if i<len(L):
        answer.extend(L[i:])
    if j<len(R):
        answer.extend(R[j:])
    return answer

In [31]:
def word_frequencies_for_file_ver6(filename):
    """
    Return alphabetically sorted list of (word,frequency) pairs 
    for the given file.
    """

    line_list = read_file(filename)
    word_list = get_words_from_line_list_ver5(line_list)
    freq_mapping = count_frequency_ver4(word_list)
    freq_mapping = merge_sort(freq_mapping)

    print("File",filename,":", end=' ')
    print(len(line_list),"lines,", end=' ')
    print(len(word_list),"words,", end=' ')
    print(len(freq_mapping),"distinct words")

    return freq_mapping

In [32]:
def ver6():
    filename_1 = "pg-grimm.txt"
    filename_2 = "pg-huckleberry_finn.txt"
    sorted_word_list_1 = word_frequencies_for_file_ver6(filename_1)
    sorted_word_list_2 = word_frequencies_for_file_ver6(filename_2)
    distance = vector_angle_ver3(sorted_word_list_1,sorted_word_list_2)
    print("The distance between the documents is: %0.6f (radians)"%distance)

In [33]:
# docdist7 - remove sorting altogether via more hashing

In [34]:
def count_frequency_ver7(word_list):
    """
    Return a dictionary mapping words to frequency.
    """
    D = {}
    for new_word in word_list:
        if new_word in D:
            D[new_word] = D[new_word]+1
        else:
            D[new_word] = 1
    return D

In [35]:
def inner_product_ver7(D1,D2):
    """
    Inner product between two vectors, where vectors
    are represented as dictionaries of (word,freq) pairs.

    Example: inner_product({"and":3,"of":2,"the":5},
                           {"and":4,"in":1,"of":1,"this":2}) = 14.0 
    """
    sum = 0.0
    for key in D1:
        if key in D2:
            sum += D1[key] * D2[key]
    return sum

In [36]:
def vector_angle_ver7(D1,D2):
    """
    The input is a list of (word,freq) pairs, sorted alphabetically.

    Return the angle between these two vectors.
    """
    numerator = inner_product_ver7(D1,D2)
    denominator = math.sqrt(inner_product_ver7(D1,D1)*inner_product_ver7(D2,D2))
    return math.acos(numerator/denominator)

In [37]:
def word_frequencies_for_file_ver7(filename):
    """
    Return alphabetically sorted list of (word,frequency) pairs 
    for the given file.
    """

    line_list = read_file(filename)
    word_list = get_words_from_line_list_ver5(line_list)
    freq_mapping = count_frequency_ver7(word_list)

    print("File",filename,":", end=' ')
    print(len(line_list),"lines,", end=' ')
    print(len(word_list),"words,", end=' ')
    print(len(freq_mapping),"distinct words")

    return freq_mapping

In [38]:
def ver7():
    filename_1 = "pg-grimm.txt"
    filename_2 = "pg-huckleberry_finn.txt"
    sorted_word_list_1 = word_frequencies_for_file_ver7(filename_1)
    sorted_word_list_2 = word_frequencies_for_file_ver7(filename_2)
    distance = vector_angle_ver7(sorted_word_list_1,sorted_word_list_2)
    print("The distance between the documents is: %0.6f (radians)"%distance)

In [39]:
# docdist8 - treat whole file as a single "line"

In [40]:
def read_file_ver8(filename):
    """ 
    Read the text file with the given filename;
    return a list of the lines of text in the file.
    """
    try:
        f = open(filename, 'r')
        return f.read()
    except IOError:
        print("Error opening or reading input file: ",filename)
        sys.exit()

In [41]:
def get_words_from_line_list_ver8(text):
    """
    Parse the given text into words.
    Return list of all words found.
    """
    text = text.translate(translation_table)
    word_list = text.split()
    return word_list

In [42]:
def word_frequencies_for_file_ver8(filename):
    """
    Return alphabetically sorted list of (word,frequency) pairs 
    for the given file.
    """

    line_list = read_file_ver8(filename)
    word_list = get_words_from_line_list_ver8(line_list)
    freq_mapping = count_frequency_ver7(word_list)

    print("File",filename,":", end=' ')
    print(len(line_list),"lines,", end=' ')
    print(len(word_list),"words,", end=' ')
    print(len(freq_mapping),"distinct words")

    return freq_mapping

In [43]:
def ver8():
    filename_1 = "pg-grimm.txt"
    filename_2 = "pg-huckleberry_finn.txt"
    sorted_word_list_1 = word_frequencies_for_file_ver8(filename_1)
    sorted_word_list_2 = word_frequencies_for_file_ver8(filename_2)
    distance = vector_angle_ver7(sorted_word_list_1,sorted_word_list_2)
    print("The distance between the documents is: %0.6f (radians)"%distance)